In [ ]:
!pip install numba
!find / -iname 'libdevice'
!find / -iname 'libnvvm.so'
!pip install mahotas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.8/dist-packages/jaxlib/cuda/nvvm/libdevice
/usr/local/cuda-11.2/nvvm-prev/libdevice
/usr/local/cuda-11.2/nvvm/libdevice
find: ‘/proc/64/task/64/net’: Invalid argument
find: ‘/proc/64/net’: Invalid argument
/usr/local/cuda-11.2/nvvm-prev/lib64/libnvvm.so
/usr/local/cuda-11.2/nvvm/lib64/libnvvm.so
find: ‘/proc/64/task/64/net’: Invalid argument
find: ‘/proc/64/net’: Invalid argument
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Loading part
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
 
downloaded = drive.CreateFile({'id': '1nEDHr7mDxV0sfnX83CujlgLIJ9C3WHK1'})   # replace the id with id of file you want to access
downloaded.GetContentFile('ExtractedRGBOulu_NPU.zip')        # replace the file name with your file
!unzip ExtractedRGBOulu_NPU

Archive:  ExtractedRGBOulu_NPU.zip
replace ExtractedRGBOulu_NPU/dev/1_1_21_1_001_rgb.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
from scipy import signal
import scipy.io
import math
import sys
import time
from IPython.display import HTML, display
from IPython.display import clear_output
import PIL
from PIL import Image
from numba import jit, njit, vectorize, cuda, uint32, f8, uint8
import mahotas
import mahotas.demos
from pylab import gray, imshow, show
import numpy as np

def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)            
    return allFiles

In [ ]:
import scipy.misc
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as tf
import skimage
import skimage.color as skcolor
import skimage.io as skio
from PIL import Image
import cv2
import numpy as np
from skimage.io import imread
from skimage.color import rgb2lab, lab2rgb
import matplotlib.pylab as plt

In [ ]:
def show(im):
    plt.imshow(im)
    plt.show()
    

In [ ]:
def rgb2ycbcr(im):
    xform = np.array([[.299, .587, .114], [-.1687, -.3313, .5], [.5, -.4187, -.0813]])
    ycbcr = im.dot(xform.T)
    ycbcr[:,:,[1,2]] += 128
    return np.uint8(ycbcr)

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

from numba import int32, float32    # import the types
from numba.experimental import jitclass


class Human:

    
    def __init__(self):
        self.img = 0
    
    
    def get_pixel(img, center, x, y):

        new_value = 0

        try:
            # If local neighbourhood pixel
            # value is greater than or equal
            # to center pixel values then
            # set it to 1
            if img[x][y] >= center:
                new_value = 1

        except:
            # Exception is required when
            # neighbourhood value of a center
            # pixel value is null i.e. values
            # present at boundaries.
            pass

        return new_value

    # Function for calculating LBP
    
    def lbp_calculated_pixel8(img, x, y):

        center = img[x][y]

        val_ar = []

        # top_left
        val_ar.append(Human.get_pixel(img, center, x - 1, y - 1))

        # top
        val_ar.append(Human.get_pixel(img, center, x - 1, y))

        # top_right
        val_ar.append(Human.get_pixel(img, center, x - 1, y + 1))

        # right
        val_ar.append(Human.get_pixel(img, center, x, y + 1))

        # bottom_right
        val_ar.append(Human.get_pixel(img, center, x + 1, y + 1))

        # bottom
        val_ar.append(Human.get_pixel(img, center, x + 1, y))

        # bottom_left
        val_ar.append(Human.get_pixel(img, center, x + 1, y - 1))

        # left
        val_ar.append(Human.get_pixel(img, center, x, y - 1))

        # Now, we need to convert binary
        # values to decimal
        power_val = [1, 2, 4, 8, 16, 32, 64, 128]

        val = 0

        for i in range(len(val_ar)):
            val += val_ar[i] * power_val[i]

        return val

    
    def rgblbp(self, img):

        # Reading the BGR image using imread() function

        strr = img

        image = cv2.imread(img)
        image = rgb2lab(image)
        height, width, xx = image.shape

        # Splitting the channels first to generate different
        # single

        # channels for merging as we don't have separate
        # channel images
        b, g, r = cv2.split(image)

        # 3 blank rgb
        img_lbpr = np.zeros((height, width), np.uint8)
        img_lbpg = np.zeros((height, width), np.uint8)
        img_lbpb = np.zeros((height, width), np.uint8)
        # # Displaying Blue channel image
        # cv2.imshow("Model Blue Image", b)

        # # Displaying Green channel image
        # cv2.imshow("Model Green Image", g)

        # # Displaying Red channel image
        # cv2.imshow("Model Red Image", r)

        # Using cv2.merge() to merge Red, Green, Blue Channels

        for i in range(0, height):
            for j in range(0, width):
                img_lbpr[i, j] = Human.lbp_calculated_pixel8(r, i, j)
                img_lbpg[i, j] = Human.lbp_calculated_pixel8(g, i, j)
                img_lbpb[i, j] = Human.lbp_calculated_pixel8(b, i, j)

        # into a coloured/multi-channeled image
        image_merge = cv2.merge([img_lbpr, img_lbpg, img_lbpb])

        path = '/content/LAB_LBP/dev'
        # path = 'D:/OpenCV/Scripts/Images'
        directory = r'/content/LAB_LBP/dev'
        os.chdir(directory)
        cv2.imwrite(strr[34:], image_merge)
#         cv2.waitKey(0)

        
        # cv2.imwrite(os.path.join(path , img[36:44]), image_merge)
        # cv2.waitKey(0)

        # save in folder dor radius=1,8 nbr



        # plt.imshow(image_merge)


        # for 16

        # print("for 16 neighbour")
        # print("..............................................................")

        # # 3 blank rgb
        # img_lbpr = np.zeros((height, width), np.uint8)
        # img_lbpg = np.zeros((height, width), np.uint8)
        # img_lbpb = np.zeros((height, width), np.uint8)

        # for i in range(0, height):
        #     for j in range(0, width):
        #         img_lbpr[i, j] = Human.lbp_calculated_pixel8(r, i, j)
        #         img_lbpg[i, j] = Human.lbp_calculated_pixel8(g, i, j)
        #         img_lbpb[i, j] = Human.lbp_calculated_pixel8(b, i, j)

        # # into a coloured/multi-channeled image
        # image_merge = cv2.merge([img_lbpr, img_lbpg, img_lbpb])

        # plt.imshow(image_merge)
        # print("for 16 neighbour")
        # print("..............................................................")


        # Displaying Merged RGB image

        # cv2.imshow("RGB_Image", image_merge)

        # # Waits for user to press any key
        # cv2.waitKey(0)


human = Human()
# split
# human.rgblbp('/content/ExtractedRGBOulu_NPU/dev/1_1_21_1_001_rgb.jpg')

In [ ]:
path_list = getListOfFiles("/content/ExtractedRGBOulu_NPU/dev")

# len(path_list)
for i in path_list:
  human.rgblbp(i)

In [ ]:
!zip -r /content/dev.zip /content/LAB_LBP/dev
from google.colab import files
files.download("/content/dev.zip")

Streaming output truncated to the last 5000 lines.
  adding: content/LAB_LBP/dev/3_1_30_4_004_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/dev/6_1_32_1_003_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/dev/4_3_25_5_012_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/dev/1_3_22_3_003_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/dev/1_3_24_2_008_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/dev/3_1_27_3_003_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/dev/6_1_35_1_006_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/dev/4_3_31_2_001_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/dev/3_2_34_1_009_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/dev/2_3_25_5_012_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/dev/5_3_23_4_011_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/dev/4_1_31_4_003_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/dev/5_3_23_3_009_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/dev/1_1_31_5_010_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/dev/4_2_33_1_

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Python program to explain Merging of Channels

# Importing cv2
import cv2
import numpy as np
from matplotlib import pyplot as plt


class Human2:

    def __init__(self):
        self.img = 0

    def get_pixel(img, center, x, y):

        new_value = 0

        try:
            # If local neighbourhood pixel
            # value is greater than or equal
            # to center pixel values then
            # set it to 1
            if img[x][y] >= center:
                new_value = 1

        except:
            # Exception is required when
            # neighbourhood value of a center
            # pixel value is null i.e. values
            # present at boundaries.
            pass

        return new_value

    # Function for calculating LBP
    def lbp_calculated_pixel8(img, x, y):

        center = img[x][y]

        val_ar = []

        # top_left
        val_ar.append(Human2.get_pixel(img, center, x - 1, y - 1))

        # top
        val_ar.append(Human2.get_pixel(img, center, x - 1, y))

        # top_right
        val_ar.append(Human2.get_pixel(img, center, x - 1, y + 1))

        # right
        val_ar.append(Human2.get_pixel(img, center, x, y + 1))

        # bottom_right
        val_ar.append(Human2.get_pixel(img, center, x + 1, y + 1))

        # bottom
        val_ar.append(Human2.get_pixel(img, center, x + 1, y))

        # bottom_left
        val_ar.append(Human2.get_pixel(img, center, x + 1, y - 1))

        # left
        val_ar.append(Human2.get_pixel(img, center, x, y - 1))

        # Now, we need to convert binary
        # values to decimal
        power_val = [1, 2, 4, 8, 16, 32, 64, 128]

        val = 0

        for i in range(len(val_ar)):
            val += val_ar[i] * power_val[i]

        return val

    def rgblbp(self, img):

        # Reading the BGR image using imread() function

        strr = img

        image = cv2.imread(img)
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
        image = rgb2lab(image)
        height, width, xx = image.shape

        # Splitting the channels first to generate different
        # single

        # channels for merging as we don't have separate
        # channel images
        b, g, r = cv2.split(image)

        # 3 blank rgb
        img_lbpr = np.zeros((height, width), np.uint8)
        img_lbpg = np.zeros((height, width), np.uint8)
        img_lbpb = np.zeros((height, width), np.uint8)
        # # Displaying Blue channel image
        # cv2.imshow("Model Blue Image", b)

        # # Displaying Green channel image
        # cv2.imshow("Model Green Image", g)

        # # Displaying Red channel image
        # cv2.imshow("Model Red Image", r)

        # Using cv2.merge() to merge Red, Green, Blue Channels

        for i in range(0, height):
            for j in range(0, width):
                img_lbpr[i, j] = Human2.lbp_calculated_pixel8(r, i, j)
                img_lbpg[i, j] = Human2.lbp_calculated_pixel8(g, i, j)
                img_lbpb[i, j] = Human2.lbp_calculated_pixel8(b, i, j)

        # into a coloured/multi-channeled image
        image_merge = cv2.merge([img_lbpr, img_lbpg, img_lbpb])

        # path = '/content/LBP_outputs/LBP_RGB/dev'
        # path = 'D:/OpenCV/Scripts/Images'
        directory = r'/content/LAB_LBP/test'
        os.chdir(directory)
        cv2.imwrite(strr[35:], image_merge)
        cv2.waitKey(0)

        
        # cv2.imwrite(os.path.join(path , img[36:44]), image_merge)
        # cv2.waitKey(0)

        # save in folder dor radius=1,8 nbr



        # plt.imshow(image_merge)


        # for 16

        # print("for 16 neighbour")
        # print("..............................................................")

        # # 3 blank rgb
        # img_lbpr = np.zeros((height, width), np.uint8)
        # img_lbpg = np.zeros((height, width), np.uint8)
        # img_lbpb = np.zeros((height, width), np.uint8)

        # for i in range(0, height):
        #     for j in range(0, width):
        #         img_lbpr[i, j] = Human.lbp_calculated_pixel8(r, i, j)
        #         img_lbpg[i, j] = Human.lbp_calculated_pixel8(g, i, j)
        #         img_lbpb[i, j] = Human.lbp_calculated_pixel8(b, i, j)

        # # into a coloured/multi-channeled image
        # image_merge = cv2.merge([img_lbpr, img_lbpg, img_lbpb])

        # plt.imshow(image_merge)
        # print("for 16 neighbour")
        # print("..............................................................")


        # Displaying Merged RGB image

        # cv2.imshow("RGB_Image", image_merge)

        # # Waits for user to press any key
        # cv2.waitKey(0)


human2 = Human2()
# split
# human.rgblbp('/content/ExtractedRGBOulu_NPU/dev/1_1_21_1_001_rgb.jpg')

In [ ]:
path_list = getListOfFiles("/content/ExtractedRGBOulu_NPU/test")

for i in path_list:
  human2.rgblbp(i)

In [ ]:
!zip -r /content/test.zip /content/LAB_LBP/test
from google.colab import files
files.download("/content/test.zip")

Streaming output truncated to the last 5000 lines.
  adding: content/LAB_LBP/test/4_3_54_3_007_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/test/6_1_45_5_007_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/test/5_3_55_4_006_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/test/2_1_39_4_008_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/test/3_1_53_4_004_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/test/5_3_38_3_005_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/test/3_1_44_4_010_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/test/3_3_45_4_005_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/test/6_3_53_2_012_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/test/3_2_54_4_001_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/test/2_1_37_2_003_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/test/2_2_50_3_001_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/test/3_3_40_2_011_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP/test/5_3_47_5_007_rgb.jpg (deflated 1%)
  adding: content/LAB_LBP

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Python program to explain Merging of Channels

# Importing cv2
import cv2
import numpy as np
from matplotlib import pyplot as plt


class Human3:

    def __init__(self):
        self.img = 0

    def get_pixel(img, center, x, y):

        new_value = 0

        try:
            # If local neighbourhood pixel
            # value is greater than or equal
            # to center pixel values then
            # set it to 1
            if img[x][y] >= center:
                new_value = 1

        except:
            # Exception is required when
            # neighbourhood value of a center
            # pixel value is null i.e. values
            # present at boundaries.
            pass

        return new_value

    # Function for calculating LBP
    def lbp_calculated_pixel8(img, x, y):

        center = img[x][y]

        val_ar = []

        # top_left
        val_ar.append(Human3.get_pixel(img, center, x - 1, y - 1))

        # top
        val_ar.append(Human3.get_pixel(img, center, x - 1, y))

        # top_right
        val_ar.append(Human3.get_pixel(img, center, x - 1, y + 1))

        # right
        val_ar.append(Human3.get_pixel(img, center, x, y + 1))

        # bottom_right
        val_ar.append(Human3.get_pixel(img, center, x + 1, y + 1))

        # bottom
        val_ar.append(Human3.get_pixel(img, center, x + 1, y))

        # bottom_left
        val_ar.append(Human3.get_pixel(img, center, x + 1, y - 1))

        # left
        val_ar.append(Human3.get_pixel(img, center, x, y - 1))

        # Now, we need to convert binary
        # values to decimal
        power_val = [1, 2, 4, 8, 16, 32, 64, 128]

        val = 0

        for i in range(len(val_ar)):
            val += val_ar[i] * power_val[i]

        return val

    def rgblbp(self, img):

        # Reading the BGR image using imread() function

        strr = img

        image = cv2.imread(img)
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
        image = rgb2lab(image)
        height, width, xx = image.shape

        # Splitting the channels first to generate different
        # single

        # channels for merging as we don't have separate
        # channel images
        b, g, r = cv2.split(image)

        # 3 blank rgb
        img_lbpr = np.zeros((height, width), np.uint8)
        img_lbpg = np.zeros((height, width), np.uint8)
        img_lbpb = np.zeros((height, width), np.uint8)
        # # Displaying Blue channel image
        # cv2.imshow("Model Blue Image", b)

        # # Displaying Green channel image
        # cv2.imshow("Model Green Image", g)

        # # Displaying Red channel image
        # cv2.imshow("Model Red Image", r)

        # Using cv2.merge() to merge Red, Green, Blue Channels

        for i in range(0, height):
            for j in range(0, width):
                img_lbpr[i, j] = Human3.lbp_calculated_pixel8(r, i, j)
                img_lbpg[i, j] = Human3.lbp_calculated_pixel8(g, i, j)
                img_lbpb[i, j] = Human3.lbp_calculated_pixel8(b, i, j)

        # into a coloured/multi-channeled image
        image_merge = cv2.merge([img_lbpr, img_lbpg, img_lbpb])

        # path = '/content/LBP_outputs/LBP_RGB/dev'
        # path = 'D:/OpenCV/Scripts/Images'
        directory = r'/content/train'
        os.chdir(directory)
        cv2.imwrite(strr[36:], image_merge)
        cv2.waitKey(0)

        
        # cv2.imwrite(os.path.join(path , img[36:44]), image_merge)
        # cv2.waitKey(0)

        # save in folder dor radius=1,8 nbr



        # plt.imshow(image_merge)


        # for 16

        # print("for 16 neighbour")
        # print("..............................................................")

        # # 3 blank rgb
        # img_lbpr = np.zeros((height, width), np.uint8)
        # img_lbpg = np.zeros((height, width), np.uint8)
        # img_lbpb = np.zeros((height, width), np.uint8)

        # for i in range(0, height):
        #     for j in range(0, width):
        #         img_lbpr[i, j] = Human.lbp_calculated_pixel8(r, i, j)
        #         img_lbpg[i, j] = Human.lbp_calculated_pixel8(g, i, j)
        #         img_lbpb[i, j] = Human.lbp_calculated_pixel8(b, i, j)

        # # into a coloured/multi-channeled image
        # image_merge = cv2.merge([img_lbpr, img_lbpg, img_lbpb])

        # plt.imshow(image_merge)
        # print("for 16 neighbour")
        # print("..............................................................")


        # Displaying Merged RGB image

        # cv2.imshow("RGB_Image", image_merge)

        # # Waits for user to press any key
        # cv2.waitKey(0)


human3 = Human3()
# split
# human.rgblbp('/content/ExtractedRGBOulu_NPU/dev/1_1_21_1_001_rgb.jpg')

In [ ]:
path_list = getListOfFiles("/content/ExtractedRGBOulu_NPU/train")

for i in path_list:
  human3.rgblbp(i)

In [ ]:
!zip -r /content/train.zip /content/train
from google.colab import files
files.download("/content/train.zip")

Streaming output truncated to the last 5000 lines.
  adding: content/train/5_2_17_4_006_rgb.jpg (deflated 1%)
  adding: content/train/5_2_20_4_002_rgb.jpg (deflated 1%)
  adding: content/train/6_2_01_3_012_rgb.jpg (deflated 1%)
  adding: content/train/3_3_09_5_004_rgb.jpg (deflated 1%)
  adding: content/train/5_1_15_5_001_rgb.jpg (deflated 1%)
  adding: content/train/4_2_12_2_002_rgb.jpg (deflated 1%)
  adding: content/train/1_1_08_5_007_rgb.jpg (deflated 1%)
  adding: content/train/3_1_18_4_006_rgb.jpg (deflated 1%)
  adding: content/train/3_3_09_4_005_rgb.jpg (deflated 1%)
  adding: content/train/1_3_07_3_015_rgb.jpg (deflated 1%)
  adding: content/train/1_3_02_4_004_rgb.jpg (deflated 1%)
  adding: content/train/5_2_04_5_007_rgb.jpg (deflated 1%)
  adding: content/train/3_2_13_2_007_rgb.jpg (deflated 1%)
  adding: content/train/1_1_01_4_005_rgb.jpg (deflated 1%)
  adding: content/train/6_2_03_5_004_rgb.jpg (deflated 1%)
  adding: content/train/3_1_16_5_008_rgb.jpg (deflated 1%)
  add

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>